In [4]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import validation_curve



In [5]:
ls


 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: AAE5-4F50

 C:\Users\hyg44\project\ML_EEG 디렉터리

2020-09-22  오후 12:18    <DIR>          .
2020-09-22  오후 12:18    <DIR>          ..
2020-09-22  오후 12:16    <DIR>          .ipynb_checkpoints
2020-09-17  오전 12:53               701 10fold_cross_validation_result.csv
2020-09-17  오전 12:53            37,055 10foldvalidation.ipynb
2020-09-17  오전 12:53            85,553 corrVI_present.ipynb
2020-09-17  오전 12:53           662,641 dataset_eeg_3.csv
2020-09-17  오전 12:53           660,057 dataset_eeg_3_O2zero.csv
2020-09-17  오전 12:53           662,602 dataset_eeg_4.csv
2020-09-17  오전 12:53         2,847,984 dataset_eeg_time_split_3.csv
2020-09-17  오전 12:53    <DIR>          EEG_data
2020-09-17  오전 12:53            33,030 EEG_ML_RESULT.ipynb
2020-09-22  오후 12:18            70,131 eeg_roc.ipynb
2020-09-17  오전 12:53            91,126 EEG_ROC.png
2020-09-17  오전 12:53    <DIR>          image
2020-09-17  오전 12:53               587 K_fold_presentation.ipynb
2020-09-17  오

In [6]:
eeg = pd.read_csv('./dataset_eeg_3.csv',index_col=0)

In [7]:
eeg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 4.5121528409392 to 0.816538269298949
Columns: 161 entries, ACh2 to task
dtypes: float64(152), object(9)
memory usage: 303.8+ KB


In [8]:
eeg.keys()

Index(['ACh2', 'ACh3', 'ACh4', 'ACh5', 'ACh6', 'ACh7', 'ACh8', 'ACh9', 'ACh10',
       'ACh11',
       ...
       'TCh25', 'TCh26', 'TCh27', 'TCh28', 'TCh29', 'TCh30', 'TCh31', 'TCh32',
       'target', 'task'],
      dtype='object', length=161)

In [9]:
eeg['target'].value_counts()

Series([], Name: target, dtype: int64)

In [10]:
eeg['task'].value_counts()

Series([], Name: task, dtype: int64)

In [11]:
state={}

state["1back_mouse"]=eeg.iloc[0:20]
state["1back_nose"]=eeg.iloc[20:40]
state["1back_O2"]=eeg.iloc[40:60]

state["2back_mouse"]=eeg.iloc[60:80]
state["2back_nose"]=eeg.iloc[80:100]
state["2back_O2"]=eeg.iloc[100:120]

state["close_rest_mouse"]=eeg.iloc[120:140]
state["close_rest_nose"]=eeg.iloc[140:160]
state["close_rest_O2"]=eeg.iloc[160:180]

state["open_rest_mouse"]=eeg.iloc[180:200]
state["open_rest_nose"]=eeg.iloc[200:220]
state["open_rest_O2"]=eeg.iloc[220:240]

In [12]:
ds_train = {}
ds_test = {}


ds_train={"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
       
          ## open+close rest m_n
          "rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]),
       
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
       "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
       "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])
       
       
      }

ds_test = {"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
           
                 ## open+close rest m_n
          "rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]),        
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
        "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
        "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])

       }




---------------------------
poster 4minuit result

-------------------------

In [13]:
eeg = pd.read_csv('./dataset_eeg_time_split_3.csv',index_col=0)

In [14]:
state={}

state["1back_mouse"]=eeg.iloc[0:80]
state["1back_nose"]=eeg.iloc[80:160]
state["1back_O2"]=eeg.iloc[160:240]

state["2back_mouse"]=eeg.iloc[240:320]
state["2back_nose"]=eeg.iloc[320:400]
state["2back_O2"]=eeg.iloc[400:480]

state["close_rest_mouse"]=eeg.iloc[480:560]
state["close_rest_nose"]=eeg.iloc[560:640]
state["close_rest_O2"]=eeg.iloc[640:720]

state["open_rest_mouse"]=eeg.iloc[720:800]
state["open_rest_nose"]=eeg.iloc[800:880]
state["open_rest_O2"]=eeg.iloc[880:960]

In [15]:
ds_train = {}
ds_test = {}


ds_train={"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
       
          ## open+close rest m_n
          "rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]),
       
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
       "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
       "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])
       
       
      }

ds_test = {"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
           
                 ## open+close rest m_n
          "rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]),        
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
        "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
        "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])

       }




### 1분씩 쪼개서 acc 출력

In [16]:
tasks = [
    ('close_rest_m_n','close_rest_O2'),
    ('close_rest_m_n','open_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2')]
num_iter=1
k_parameter=10

for mode in ['lda']:
    for task_train,task_test in tasks:

        print('-------------------')
        print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
        train_4 = ds_train[task_train]
        test_4  = ds_test[task_test]
        
        for condition in range(4):
            train_3=train_4[train_4['time']==condition+1]
            test_3=test_4[test_4['time']==condition+1]

            X = np.array(train_3.iloc[:,0:-4])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            dflist=[]
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
    #                         X_final = X
                        X_final = lda.transform(X)


                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-4])
                    if mode == 'lda':
    #                         pass
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa==0)
                    Nose = Nose+ np.count_nonzero(aaa==1)
            print('---------------------------')
            print('time:{}th minute'.format(condition+1))
            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))
        print("\n")

-------------------
training:close_rest_m_n/test:close_rest_O2/mode:lda
---------------------------
time:1th minute
model_score:0.875
mouse:99(0.49), nose:101(0.51) / total:200
---------------------------
time:2th minute
model_score:0.875
mouse:99(0.49), nose:101(0.51) / total:200
---------------------------
time:3th minute
model_score:0.8
mouse:111(0.56), nose:89(0.45) / total:200
---------------------------
time:4th minute
model_score:0.875
mouse:88(0.44), nose:112(0.56) / total:200


-------------------
training:close_rest_m_n/test:open_rest_O2/mode:lda
---------------------------
time:1th minute
model_score:0.9
mouse:40(0.2), nose:160(0.8) / total:200
---------------------------
time:2th minute
model_score:0.875
mouse:109(0.55), nose:91(0.46) / total:200
---------------------------
time:3th minute
model_score:0.8
mouse:129(0.65), nose:71(0.35) / total:200
---------------------------
time:4th minute
model_score:0.875
mouse:68(0.34), nose:132(0.66) / total:200


-------------------
t

In [ ]:
def LDA_RF_M_N_O2 (tasks, k_parameter, num_iter):

    for mode in ['lda']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]
    
            X = np.array(train_3.iloc[:,0:-4])
            Y = np.array(train_3['target'])
            Y_idx=np.array(train_3['time'])
            groups = np.array(train_3['subject'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            dflist=[]
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):
                    print("Train:1.",Y[train_index],"\n",
                          "2.",Y_idx[train_index],"\n",
                          =groups[train_index]

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
#                         X_final = X
                        X_final = lda.transform(X)
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-4])
                    if mode == 'lda':
#                         pass
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa==0)
                    Nose = Nose+ np.count_nonzero(aaa==1)

            df=pd.DataFrame(dflist)
            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))

In [ ]:
# Stratified K fold
print("Poster Result data")
tasks1 = [
    ('close_rest_m_n','close_rest_O2'),
    ('close_rest_m_n','open_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2')]

print("LDA-RF")
LDA_RF_M_N_O2(tasks1,k_parameter = 10,num_iter = 1)

### GroupKFold

In [74]:
import numpy as np
from sklearn.model_selection import GroupKFold, GroupShuffleSplit,LeavePGroupsOut

tasks=[
    ('close_rest_m_n','close_rest_O2')]

for task_train,task_test in tasks:
    train_3 = ds_train[task_train]
    
    X = np.array(train_3.iloc[:,0:-4])
    y=np.array(train_3['time'])
    y2=np.array(train_3['target'])
    groups=np.array(train_3['subject'])
 
    
    gss = GroupShuffleSplit(n_splits=10,random_state=42)
    gkf = GroupKFold(n_splits=20)
    lpg=LeavePGroupsOut(n_groups=10)
    gss.get_n_splits(X, y, groups)
    
    for train_index, test_index in gss.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        print("Train:1.",y[train_index],"\n",
              "2.",y2[train_index],"\n",
              "3.",y3[train_index],"\n",
              "Test:1.",y[test_index],"\n",
              "2.",y2[test_index],"\n",
              "3.",y3[test_index],"\n")

Train:1. [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3] 
 2. [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 
 3. [ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 17. 19. 20.  3.  4.
  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 17. 19. 20.  3.  4.  5.  6.
  7.  8.  9. 10. 11. 12. 13. 14. 15. 17. 19. 20.  3.  4.  5.  6.  7.  8.
  9. 10. 11. 12. 13. 14. 15. 17. 19. 20.  3.  4.  5.  6.  7.  8.  9. 10.
 11. 12. 13. 14. 15. 17. 19. 20.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.
 13. 14. 15. 17. 19. 20.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.
 15. 17. 19. 20.  3. 

In [77]:
#Group K fold
print("Poster Result data")
tasks1 = [
    ('close_rest_m_n','close_rest_O2'),
    ('close_rest_m_n','open_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2')]

print("LDA-RF")
LDA_RF_M_N_O2(tasks1,k_parameter = 10,num_iter = 1)

Poster Result data
LDA-RF
-------------------
training:close_rest_m_n/test:close_rest_O2/mode:lda
model_score:0.996875
mouse:450(0.56), nose:350(0.44) / total:800
-------------------
training:close_rest_m_n/test:open_rest_O2/mode:lda
model_score:0.996875
mouse:371(0.46), nose:429(0.54) / total:800
-------------------
training:close_rest_m_n/test:1back_O2/mode:lda
model_score:0.996875
mouse:251(0.31), nose:549(0.69) / total:800
-------------------
training:close_rest_m_n/test:2back_O2/mode:lda
model_score:0.996875
mouse:340(0.42), nose:460(0.57) / total:800


--- 
### POSTER RESULT

---

In [ ]:
def LDA_RF_M_N_O2 (tasks, k_parameter, num_iter):

    for mode in ['lda']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-2])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
#                         X_final = X
                        X_final = lda.transform(X)
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-2])
                    if mode == 'lda':
#                         pass
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa==0)
                    Nose = Nose+ np.count_nonzero(aaa==1)

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))

In [ ]:
print("Poster Result data")
tasks1 = [
    ('close_rest_m_n','close_rest_O2'),
    ('close_rest_m_n','open_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2'),

 ]
print("LDA-RF")
LDA_RF_M_N_O2(tasks1,k_parameter = 10,num_iter = 1)


---
---

In [349]:
def LDARF (tasks, k_parameter, num_iter):

#     for mode in ['no_lda','lda']:
    for mode in ['lda']:
        for task_train,task_test in tasks:
            print('==========================================================================')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-2])
            Y = np.array(train_3['target'])
            Y_idx = np.array(train_3['task'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
            lda = LinearDiscriminantAnalysis(n_components=1)
            all_scores = []
            correct_Mouse = 0
            correct_Nose = 0
            predict_Mouse = 0
            predict_Nose = 0
            df2 = pd.DataFrame(data = {'predict_value','true_value','true_task'})
            
            lda.fit(X,Y)
            X_final = lda.transform(X)
            

            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y_idx):                    
                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)
#                     print('-------------------')
#                     print('test_index_num:{}'.format(test_index))
#                     print('1sub_set_predict_value:{}'.format(rnd_clf.predict(X_final[test_index])))
#                     print('lsub_set_true_value:{}'.format(Y[test_index]))
#                     print('-------------------')
                    test = np.array(test_3.iloc[:,0:-2])
                    test_Y = np.array(test_3['target'])
                    test_Y_idx = np.array(test_3['task'])
                    test = lda.transform(test)

                    aaa = rnd_clf.predict(test)
                    
                    
                    predict_Mouse = predict_Mouse+np.count_nonzero(aaa==0)
                    predict_Nose = predict_Nose+ np.count_nonzero(aaa==1)
                    correct_Mouse = correct_Mouse +np.count_nonzero(test_Y==0)
                    correct_Nose = correct_Nose +np.count_nonzero(test_Y==1)
                    
                    df = pd.DataFrame(data = {'predict_value':aaa,'true_value':test_Y,'true_task':test_Y_idx})
#                     print('------------------------------------------')
#                     print('test_data_predict_value')
#                     print(df['predict_value'].value_counts())
#                     print('test_data_true_value')
#                     print(df['true_value'].value_counts())
#                     print('test_data_task')
#                     print(df['true_task'].value_counts())
#                     print('------------------------------------------')

                    df2 = df2.append(df)
            
#             print('------------------------------------------')
#             print('test_data_predict_value')
#             print(df2['predict_value'].value_counts())
#             print('test_data_true_value')
#             print(df2['true_value'].value_counts())
#             print('test_data_task')
#             print(df2['true_task'].value_counts())
#             print('------------------------------------------')

            print('subset_accuracy:{}'.format(all_scores))
            print('model_score:{}'.format(np.mean(all_scores)))
            print('predict_mouse:{}, predict_nose:{} / predict_total:{}'.format(predict_Mouse,predict_Nose,(predict_Mouse+predict_Nose)))
            print('correct_mouse:{}, correct_nose:{} / correct_total:{}'.format(correct_Mouse,correct_Nose,(correct_Mouse+correct_Nose)))


In [350]:
tasks1 = [
    ('rest_m_n','open_rest_O2'),
    ('rest_m_n','close_rest_O2'),
    
    
     ('rest_m_n','1back_nose'),
    ('rest_m_n','1back_mouse'),
    ('rest_m_n','1back_O2'),
    
     ('rest_m_n','2back_nose'),
    ('rest_m_n','2back_mouse'),
    ('rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)



LDARF
training:rest_m_n/test:open_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
0.0    140
1.0     60
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_open_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:140, predict_nose:60 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:rest_m_n/test:close_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
1.0    110
0.0     90
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_close_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:90, predict_nose:110 / pr

In [351]:
tasks1 = [
    ('close_rest_m_n','open_rest_O2'),
    ('close_rest_m_n','close_rest_O2'),
    
    
     ('close_rest_m_n','1back_nose'),
    ('close_rest_m_n','1back_mouse'),
    ('close_rest_m_n','1back_O2'),
    
     ('close_rest_m_n','2back_nose'),
    ('close_rest_m_n','2back_mouse'),
    ('close_rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)


LDARF
training:close_rest_m_n/test:open_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
1.0    100
0.0    100
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_open_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 0.75, 1.0, 0.75, 1.0, 1.0]
model_score:0.95
predict_mouse:100, predict_nose:100 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:close_rest_m_n/test:close_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
0.0    101
1.0     99
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_close_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.75]
model_score:0.95
predict_mouse:101, p

In [352]:
tasks1 = [
    ('open_rest_m_n','open_rest_O2'),
    ('open_rest_m_n','close_rest_O2'),
    
    
     ('open_rest_m_n','1back_nose'),
    ('open_rest_m_n','1back_mouse'),
    ('open_rest_m_n','1back_O2'),
    
     ('open_rest_m_n','2back_nose'),
    ('open_rest_m_n','2back_mouse'),
    ('open_rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)


LDARF
training:open_rest_m_n/test:open_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
0.0    120
1.0     80
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_open_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[1.0, 0.75, 0.75, 1.0, 0.75, 1.0, 1.0, 0.75, 1.0, 0.75]
model_score:0.875
predict_mouse:120, predict_nose:80 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:open_rest_m_n/test:close_rest_O2/mode:lda
------------------------------------------
test_data_predict_value
0.0    159
1.0     41
Name: predict_value, dtype: int64
test_data_true_value
0.0    200
Name: true_value, dtype: int64
test_data_task
rest_close_O2    200
Name: true_task, dtype: int64
------------------------------------------
subset_accuracy:[0.5, 1.0, 0.5, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:0.875
predict_mouse:159, 

In [353]:
def LDARF (tasks, k_parameter, num_iter):

#     for mode in ['no_lda','lda']:
    for mode in ['lda']:
        for task_train,task_test in tasks:
            print('==========================================================================')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-2])
            Y = np.array(train_3['target'])
            Y_idx = np.array(train_3['task'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
            lda = LinearDiscriminantAnalysis(n_components=1)
            all_scores = []
            correct_Mouse = 0
            correct_Nose = 0
            predict_Mouse = 0
            predict_Nose = 0
            df2 = pd.DataFrame(data = {'predict_value','true_value','true_task'})
            
            lda.fit(X,Y)
            X_final = lda.transform(X)
            

            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y_idx):                    
                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)
#                     print('-------------------')
#                     print('test_index_num:{}'.format(test_index))
#                     print('1sub_set_predict_value:{}'.format(rnd_clf.predict(X_final[test_index])))
#                     print('lsub_set_true_value:{}'.format(Y[test_index]))
#                     print('-------------------')
                    test = np.array(test_3.iloc[:,0:-2])
                    test_Y = np.array(test_3['target'])
                    test_Y_idx = np.array(test_3['task'])
                    test = lda.transform(test)

                    aaa = rnd_clf.predict(test)
                    
                    
                    predict_Mouse = predict_Mouse+np.count_nonzero(aaa==0)
                    predict_Nose = predict_Nose+ np.count_nonzero(aaa==1)
                    correct_Mouse = correct_Mouse +np.count_nonzero(test_Y==0)
                    correct_Nose = correct_Nose +np.count_nonzero(test_Y==1)
                    
                    df = pd.DataFrame(data = {'predict_value':aaa,'true_value':test_Y,'true_task':test_Y_idx})
#                     print('------------------------------------------')
#                     print('test_data_predict_value')
#                     print(df['predict_value'].value_counts())
#                     print('test_data_true_value')
#                     print(df['true_value'].value_counts())
#                     print('test_data_task')
#                     print(df['true_task'].value_counts())
#                     print('------------------------------------------')

                    df2 = df2.append(df)
            
#             print('------------------------------------------')
#             print('test_data_predict_value')
#             print(df2['predict_value'].value_counts())
#             print('test_data_true_value')
#             print(df2['true_value'].value_counts())
#             print('test_data_task')
#             print(df2['true_task'].value_counts())
#             print('------------------------------------------')

            print('subset_accuracy:{}'.format(all_scores))
            print('model_score:{}'.format(np.mean(all_scores)))
            print('predict_mouse:{}, predict_nose:{} / predict_total:{}'.format(predict_Mouse,predict_Nose,(predict_Mouse+predict_Nose)))
            print('correct_mouse:{}, correct_nose:{} / correct_total:{}'.format(correct_Mouse,correct_Nose,(correct_Mouse+correct_Nose)))


In [354]:
tasks1 = [
    ('rest_m_n','open_rest_O2'),
    ('rest_m_n','close_rest_O2'),
    
    
     ('rest_m_n','1back_nose'),
    ('rest_m_n','1back_mouse'),
    ('rest_m_n','1back_O2'),
    
     ('rest_m_n','2back_nose'),
    ('rest_m_n','2back_mouse'),
    ('rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)



LDARF
training:rest_m_n/test:open_rest_O2/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:140, predict_nose:60 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:rest_m_n/test:close_rest_O2/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:90, predict_nose:110 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:rest_m_n/test:1back_nose/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:120, predict_nose:80 / predict_total:200
correct_mouse:0, correct_nose:200 / correct_total:200
training:rest_m_n/test:1back_mouse/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:1.0
predict_mouse:150, predict_nose:50 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:rest_m_n/test:1back_O2/mode:lda
subset_

In [355]:
tasks1 = [
    ('close_rest_m_n','open_rest_O2'),
    ('close_rest_m_n','close_rest_O2'),
    
    
     ('close_rest_m_n','1back_nose'),
    ('close_rest_m_n','1back_mouse'),
    ('close_rest_m_n','1back_O2'),
    
     ('close_rest_m_n','2back_nose'),
    ('close_rest_m_n','2back_mouse'),
    ('close_rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)


LDARF
training:close_rest_m_n/test:open_rest_O2/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 0.75, 0.5, 1.0, 1.0, 0.75, 1.0, 1.0]
model_score:0.9
predict_mouse:100, predict_nose:100 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:close_rest_m_n/test:close_rest_O2/mode:lda
subset_accuracy:[0.75, 1.0, 1.0, 1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0]
model_score:0.95
predict_mouse:101, predict_nose:99 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:close_rest_m_n/test:1back_nose/mode:lda
subset_accuracy:[1.0, 0.75, 1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0]
model_score:0.95
predict_mouse:41, predict_nose:159 / predict_total:200
correct_mouse:0, correct_nose:200 / correct_total:200
training:close_rest_m_n/test:1back_mouse/mode:lda
subset_accuracy:[1.0, 0.75, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.75]
model_score:0.9
predict_mouse:62, predict_nose:138 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:clos

In [356]:
tasks1 = [
    ('open_rest_m_n','open_rest_O2'),
    ('open_rest_m_n','close_rest_O2'),
    
    
     ('open_rest_m_n','1back_nose'),
    ('open_rest_m_n','1back_mouse'),
    ('open_rest_m_n','1back_O2'),
    
     ('open_rest_m_n','2back_nose'),
    ('open_rest_m_n','2back_mouse'),
    ('open_rest_m_n','2back_O2'),

 ]
print("LDARF")
LDARF(tasks1,k_parameter = 10,num_iter = 1)


LDARF
training:open_rest_m_n/test:open_rest_O2/mode:lda
subset_accuracy:[1.0, 1.0, 1.0, 0.5, 0.75, 1.0, 1.0, 1.0, 1.0, 0.5]
model_score:0.875
predict_mouse:120, predict_nose:80 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:open_rest_m_n/test:close_rest_O2/mode:lda
subset_accuracy:[1.0, 0.75, 1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 0.5, 0.75]
model_score:0.875
predict_mouse:159, predict_nose:41 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:open_rest_m_n/test:1back_nose/mode:lda
subset_accuracy:[1.0, 0.75, 0.75, 1.0, 0.75, 0.75, 1.0, 1.0, 0.75, 1.0]
model_score:0.875
predict_mouse:91, predict_nose:109 / predict_total:200
correct_mouse:0, correct_nose:200 / correct_total:200
training:open_rest_m_n/test:1back_mouse/mode:lda
subset_accuracy:[0.75, 1.0, 1.0, 1.0, 1.0, 0.75, 0.75, 1.0, 1.0, 0.75]
model_score:0.9
predict_mouse:84, predict_nose:116 / predict_total:200
correct_mouse:200, correct_nose:0 / correct_total:200
training:

In [1]:
Y[test_index]

NameError: name 'Y' is not defined